# 04 - Multiple logistic regression

## Data

Source of data: Titanic

www.kaggle.com

https://www.kaggle.com/datasets/hesh97/titanicdataset-traincsv 

Dataset titanic.csv

In [63]:
library(readr)
titanic <- read_csv("data/titanic.csv",
                 show_col_types = FALSE)
titanic$Pclass <- as.factor(titanic$Pclass)
titanic$Sex <- as.factor(titanic$Sex)

head(titanic)


PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
<dbl>,<dbl>,<fct>,<chr>,<fct>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>
1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NA,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NA,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NA,S
6,0,3,"Moran, Mr. James",male,NA,0,0,330877,8.4583,NA,Q


## SAS program snippet

The following SAS code will be executed.

The option descending reverses the order of the levels in the dependent variable.

## Results

The output is divided into blocks to explain it and to reproduce it afterwards in the different languages.

The sequence and contents of the blocks differes between simple and multiple logistic regression.

The occurrence of categorical variables also adds other blocks.

### Block 1
![Block 1](img_screenshots/block_1.png)

Row 1 refers to the dataset which was used in this procedure.

Row 2 gives the response variable or dependent variable for the logistic regression.

Row 3 gives the number of response levels equal to the available levels of the dependent variable in the dataset.

Row 4 names the type of model. In this case it is a logistic regression or binary logit as stated here.

Row 5 gives the name of the optimization technique which was used. Here is a source for differences between the statistical programs.
In SAS, the default method is Fisher’s scoring method.
In R, the glm documentation mentions iteratively reweighted least squares (IWLS) as the method.
In Stata, it is the Newton-Raphson algorithm. 
These are the three main methods.

You have to look into the small print in the description of the method.

### R chunk for reproduction

In [64]:
library(broom) 
my_logistic <- glm(Survived ~ Sex + Pclass + Age, data = titanic, family = binomial)

summary(my_logistic)

# Number of response levels
table("survived" = titanic$Survived)


Call:
glm(formula = Survived ~ Sex + Pclass + Age, family = binomial, 
    data = titanic)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  3.777013   0.401123   9.416  < 2e-16 ***
Sexmale     -2.522781   0.207391 -12.164  < 2e-16 ***
Pclass2     -1.309799   0.278066  -4.710 2.47e-06 ***
Pclass3     -2.580625   0.281442  -9.169  < 2e-16 ***
Age         -0.036985   0.007656  -4.831 1.36e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 964.52  on 713  degrees of freedom
Residual deviance: 647.28  on 709  degrees of freedom
  (177 Beobachtungen als fehlend gelöscht)
AIC: 657.28

Number of Fisher Scoring iterations: 5


survived
  0   1 
549 342 

Data set will be part of the result of the summary() function.

The response variable is given in the formula.

The number of levels can be obtained with the table() function. SAS handles character values, R needs numeric values for the response variable.

The summary() function also provides the information that Fisher Scoring was used.

### Block 2
![Block 2](img_screenshots/block_2.png)

The number of observations used might be less than the number of observations read. SAS performs a listwise deletion (complete case analysis) if missing values are present.

### R chunk for reproduction

In [65]:
nrow(titanic)
nobs(my_logistic)
nrow(titanic) - nobs(my_logistic)

[1] 891

[1] 714

[1] 177

The number of observations is the result of the nrow() function.

The number of observations used is the result of the nobs() function.


### Block 3
![Block 3](img_screenshots/block_3.png)

The levels and the frequencies for the dependent variable are provided here.

It is also stated which probability is modeled here. The order was reversed here with the descending option in the proc logistic statement.

By default SAS models the 0 while other statistical programs model the 1. 
Categorical levels would be sorted in alphabetical order and the first level would be modeled.

### R chunk for reproduction

In [66]:
suppressPackageStartupMessages(library(tidyverse))
titanic %>%
group_by(Survived) %>%
count(name = "Total Frequency")

Survived,Total Frequency
<dbl>,<int>
0,549
1,342


### Block 4
![Block 4](img_screenshots/block_4.png)

The important information that the model converged can be found here.

Coding of categorical is listed here.

This coding might differ from the coding in other statistic programming languages.


## R chunk for reproduction

In [67]:
contrasts(titanic$Sex)

contrasts(titanic$Pclass)


,male
female,0
male,1


,2,3
1,0,0
2,1,0
3,0,1


R and SAS have a different default coding.

R uses dummy coding.

SAS uses sum coding.

### Block 5
![Block 5](img_screenshots/block_5.png)

The model fit status is described by 
-  AIC (Akaike Information Criterion): Smaller is better.
-  SC (Schwarz Criterion): Smaller is better.
-  -2 Log L (negative two times the log-likelihood)



### R chunk for reproduction

In [68]:
glance(my_logistic)

null.deviance,df.null,logLik,AIC,BIC,deviance,df.residual,nobs
<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
964.516,713,-323.6416,657.2831,680.1375,647.2831,709,714


null.deviance and df.null refer to the intercept only model.

BIC (Bayes information criterion) is also SC (Schwarz criterion).

you can run an intercept model with the following code.

In [69]:
my_logistic_intercept <- glm(Survived ~ 1, 
                             data = titanic %>% select(Survived, Age, Sex, Pclass) %>% na.omit(), 
                             family = binomial)
glance(my_logistic_intercept)


null.deviance,df.null,logLik,AIC,BIC,deviance,df.residual,nobs
<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
964.516,713,-482.258,966.516,971.0868,964.516,713,714


The values are similiar to the SAS output.

### Block 6
![Block 6](img_screenshots/block_6.png)

These global tests test the null hypothesis that all regression coefficents are zero.

The tests are different chi-square tests.


### R chunk for reproduction

TODO: Here is a lot to do for reproduction. There are some differences for the tests.

#### Globaltest

In [70]:
#if (!require("BiocManager", quietly = TRUE))
#install.packages("BiocManager")
#BiocManager::install(version = "3.18")
#library(BiocManager)
#BiocManager::install("globaltest")library(globaltest) # from Bioconductor
gt(Survived ~ 1, Survived ~ Age + Sex + Pclass, data = titanic)
   


  p-value Statistic Expected Std.dev #Cov
1  0.0217     0.588    0.112   0.158    6

#### Likely ratio test

In [71]:
library(lmtest)
lrtest(my_logistic_intercept, my_logistic)

,#Df,LogLik,Df,Chisq,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,-482.2580,NA,NA,NA
2,5,-323.6416,4,317.2328,2.074158e-67


In [72]:
# z-test
# Wald’s chi-squared statistic for the hypothesis that the coefficient of glucose
# is zero, or equivalently that the odds-ratio is one
m0 <- glm(Survived ~ 1, data = titanic, family = binomial)
m1 <- glm(Survived ~ Age + Sex + Pclass, data = titanic, family = binomial)
b <- coef(m1)
b
se <- sqrt(diag(vcov(m1)))
se

(Intercept)         Age     Sexmale     Pclass2     Pclass3 
 3.77701265 -0.03698527 -2.52278092 -1.30979927 -2.58062532

(Intercept)         Age     Sexmale     Pclass2     Pclass3 
0.401123305 0.007655948 0.207390924 0.278065527 0.281442020

In [73]:
# z-test
# Wald’s chi-squared statistic for the hypothesis that the coefficient of glucose
# is zero, or equivalently that the odds-ratio is one
m0 <- glm(Survived ~ 1, data = titanic, family = binomial)
m1 <- glm(Survived ~ Age + Sex + Pclass, data = titanic, family = binomial)

b <- coef(m1)
b
se <- sqrt(diag(vcov(m1)))
se
(b[2]/se[2])^2

# likeli ratio test
#-2*(logLik(m0) - logLik(m1))
-2*(logLik(m0) - logLik(m1))

(Intercept)         Age     Sexmale     Pclass2     Pclass3 
 3.77701265 -0.03698527 -2.52278092 -1.30979927 -2.58062532

(Intercept)         Age     Sexmale     Pclass2     Pclass3 
0.401123305 0.007655948 0.207390924 0.278065527 0.281442020

Age 
23.33778

'log Lik.' 539.372 (df=1)

### Block 7
![Block 7](img_screenshots/block_7.png)

Column 1 "Effect" lists the variables which are interpreted by the point estimate.

Column 2 "Point Estimate" is interpreted as an odds ratio. 
One unit change in the independent variable changes the probability for the modelled event by the estimated value.

Column 3 and 4 give the confidence interval for the odds ratio.

### R chunk for reproduction

Here is a lot to do for reproduction.

In [74]:
(b[2]/se[2])^2
(b[3]/se[3])^2
(b[4]/se[4])^2
(b[5]/se[5])^2

Age 
23.33778

Sexmale 
147.972

Pclass2 
22.18785

Pclass3 
 84.076

### Block 8
![Block 8](img_screenshots/block_8.png)

Column 1 "Parameter" lists the intercept and the parameter in the model.

Column 2 "DF" gives the degrees of freedom for every parameter.

Column 3 "Estimate" lists the logit regression estimates for every parameter given that the other parameter are held constant. 

$log(p / (1 - p)) = -6.10 + 0.04 * glucose$ with p as the probability for diabetes.

Column 4 "Standard Error" gives the standard errors of the individual regression coefficients.

Column 5 "Wald Chi-Square" tests the null hypothesis that the regression coefficient is zero given that the other predictors are in the model.

Column 6 "Pr > ChiSq" gives the p-value for the Wald Chi-Square statistic.

### R chunk for reproduction

In [75]:
tidy(my_logistic)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),3.77701265,0.401123305,9.416089,4.682044e-21
Sexmale,-2.52278092,0.207390924,-12.164375,4.811152e-34
Pclass2,-1.30979927,0.278065527,-4.710398,2.472337e-06
Pclass3,-2.58062532,0.281442020,-9.169296,4.761161e-20
Age,-0.03698527,0.007655948,-4.830919,1.359041e-06


The estimates here are different from the SAS estimates because the reference levels are different by default.

![Block 8a](img_screenshots/block_11.png)

The block above is a result of the SAS code below with a changed coding of class variables.

The coding in R can be changed manually.

In [76]:
titanic_relev <- titanic
titanic_relev$Sexfemale <- ifelse(titanic_relev$Sex == "female", 1, -1)
titanic_relev$Pclass1 <- (titanic_relev$Pclass == "1") * 1 + (titanic_relev$Pclass == "2") * 0 + (titanic_relev$Pclass == "3") * (-1)
titanic_relev$Pclass2 <- (titanic_relev$Pclass == "1") * 0 + (titanic_relev$Pclass == "2") * 1 + (titanic_relev$Pclass == "3") * (-1)

my_logistic_relev <- glm(Survived ~ Age + Sexfemale + Pclass1 + Pclass2, data = titanic_relev, family = binomial)
tidy(my_logistic_relev)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.21881400,0.257562502,4.73210964,2.221983e-06
Age,-0.03698527,0.007655948,-4.83091929,1.359041e-06
Sexfemale,1.26139046,0.103695462,12.16437473,4.811152e-34
Pclass1,1.29680820,0.167829578,7.72693475,1.101671e-14
Pclass2,-0.01299107,0.147025941,-0.08835905,9.295913e-01


Changing the coding results in similar results as in the SAS output.

### Block 9
![Block 9](img_screenshots/block_9.png)

Column 1 "Effect" lists the variables which are interpreted by the point estimate.

Column 2 "Point Estimate" is interpreted as an odds ratio. 
One unit change in the independent variable changes the probability for the modelled event by the estimated value.

Column 3 and 4 give the confidence interval for the odds ratio.


## R code for reproduction

TODO: Here is a lot to do for reproduction. There is a diffence to the SAS output for categorical variables.

In [77]:
exp(coef(my_logistic_relev))

(Intercept)         Age   Sexfemale     Pclass1     Pclass2 
  3.3831729   0.9636903   3.5303269   3.6576037   0.9870929

In [78]:
suppressMessages(exp(confint(my_logistic_relev)))

,2.5 %,97.5 %
(Intercept),2.0589570,5.6604961
Age,0.9490535,0.9780124
Sexfemale,2.8929687,4.3463297
Pclass1,2.6493306,5.1202648
Pclass2,0.7386874,1.3155812


### Block 10
![Block 10](img_screenshots/block_10.png)

These parameter describe the association between the predicted probabilities and observed responses.


## R code for reproduction

In [79]:
library(survival)
concordance(my_logistic)

Call:
concordance.lm(object = my_logistic)

n= 714 
Concordance= 0.8523 se= 0.01549
concordant discordant     tied.x     tied.y    tied.xy 
    104647      18004        309     130627        954 

In [85]:
library(yardstick)
obs <- my_logistic$model$Survived
pred <- as.integer(round(predict(my_logistic, type = "response")))
str(obs)
str(pred)
df1 <- table(obs, pred)

 num [1:714] 0 1 1 1 0 0 0 1 1 1 ...
 int [1:714] 0 1 1 1 0 0 0 1 1 1 ...


In [86]:
cf <- conf_mat(df1, truth = obs, estimate = pred)
summary(cf)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.7885154
kap,binary,0.5579763
sens,binary,0.8109339
spec,binary,0.7527273
ppv,binary,0.8396226
npv,binary,0.7137931
mcc,binary,0.5585150
j_index,binary,0.5636612
bal_accuracy,binary,0.7818306
